In [9]:
from tensorflow.examples.tutorials.mnist import input_data
import os
print os.getcwd()

root = "_tensorflow/MNIST_data/"
mnist = input_data.read_data_sets(root, one_hot=True)


print "Trainning data size: ", mnist.train.num_examples
print "Validating data size: ", mnist.validation.num_examples
print "Testing data size: ", mnist.test.num_examples


print "Example training data: ", mnist.train.num_examples
print "Example training data label: ", mnist.train.labels[0]

/Users/zhangzhen/gitRepository/AnalyticsVidhya
Extracting _tensorflow/MNIST_data/train-images-idx3-ubyte.gz


Extracting _tensorflow/MNIST_data/train-labels-idx1-ubyte.gz
Extracting _tensorflow/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting _tensorflow/MNIST_data/t10k-labels-idx1-ubyte.gz


Trainning data size:  55000
Validating data size:  5000
Testing data size:  10000
Example training data:  55000
Example training data label:  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [11]:
batch_size = 1000
xs, ys = mnist.train.next_batch(batch_size)
print "X shape: ", xs.shape
print "Y shape: ", ys.shape

X shape:  (1000, 784)
Y shape:  (1000, 10)


In [14]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

INPUT_NODE = 784
OUTPUT_NODE = 10

LAYER1_NODE = 500
BATCH_SZIE = 100

LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99

REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99


def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    if avg_class is None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        return tf.matmul(layer1, weights1) + biases2
    else:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)
        

def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    
    weights1 = tf.Variable([INPUT_NODE, LAYER1_NODE], stddev=0.1)
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))
    
    y = inference(x, None, weights1, biases1, weights2, biases2)
    global_step = tf.Variable(0, trainable=False)
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    average_y = inference(x, variables_averages_op, weights1, biases1, weights2, biases2)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(y, tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularization = regularizer(weights1) + regularizer(weights2)
    
    loss = cross_entropy_mean + regularization
    
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SZIE,
        LEARNING_RATE_DECAY
    )
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')
    
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
